Specify where your evaluation results are saved

In [2]:
path_to_results = f"./llm_prompting_evaluation_results/VAGOsolutions/SauerkrautLM-Mixtral-8x7B-Instruct/"

In [3]:
mapping = {'precision_macro':'Macro-P', 
           'precision_micro':'Micro-P',
           'recall_macro':'Macro-R', 
           'recall_micro':'Micro-R', 
           'f1_macro':'Macro-F1', 
           'f1_micro':'Micro-F1', 
           'prompt_1':'1', 
           'prompt_2':'2', 
           'prompt_3':'3',
           'prompt_4':'4',
           'kriterium':'AC',
          'gewichtung':'W',
          'maxPunkte':'MNP',
          'zkNummer':'ID'}
prompt_id_list = ['prompt_1', 'prompt_2', 'prompt_3', 'prompt_4']
averages = ['macro', 'micro']
scores = ['precision', 'recall', 'f1']



In [5]:
from ast import literal_eval
import json as js
import re
from statistics import harmonic_mean, mean
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, matthews_corrcoef
from typing import Literal
import sys

from training_data_handler import TrainingDataHandler

tdh = TrainingDataHandler()


ModuleNotFoundError: No module named 'training_data_handler'

In [ ]:
dataset = tdh.get_criteria_relevance_training_set()

Distribution of labels for train set.
             label  text
0     has_criteria    32
1  has_no_criteria    32
Distribution of labels for validation set.
             label  text
0     has_criteria    66
1  has_no_criteria    63
Distribution of labels for test set.
             label  text
0     has_criteria    66
1  has_no_criteria    64


In [ ]:
# Function to highlight the max value in each column, but only for numeric columns
def highlight_max_numeric(s):
    '''
    Highlight the maximum in a Series yellow if it's numeric.
    '''
    if s.dtype in ['int', 'float']:
        is_max = s == s.max()
        return ['background-color: yellow' if v else '' for v in is_max]
    else:
        return ['' for _ in s]
    
def bold_max_values(data):
    # Create a copy of the DataFrame to avoid modifying the original data
    df_temp = data.copy()
    for column in df_temp.columns:
        if pd.api.types.is_numeric_dtype(df_temp[column]):
            # Find the maximum value in the column
            max_value = df_temp[column].max()
            # Apply LaTeX bold formatting to the max value
            df_temp[column] = df_temp[column].apply(lambda x: f'\\textbf{{{x}}}' if x == max_value else x)
    
    columns = df_temp.columns.tolist()
    for x in columns:
        df_temp.rename(columns={x: f'\\textbf{{{x}}}'}, inplace=True)
    return df_temp


def make_columns_bold(data):
    
    df_temp = data.copy()
    
    columns = df_temp.columns.tolist()
    for x in columns:
        df_temp.rename(columns={x: f'\\textbf{{{x}}}'}, inplace=True)
    return df_temp
    



# Detecting binary criteria presence

In [ ]:
def get_result_table(y_true, y_pred, split):
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_micro = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    precision_macro = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
    precision_micro = precision_score(y_true=y_true, y_pred=y_pred, average='micro')
    recall_macro = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
    recall_micro = precision_score(y_true=y_true, y_pred=y_pred, average='micro')
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    mcc = matthews_corrcoef(y_true=y_true, y_pred=y_pred)

    classificatio_results = list()
    classificatio_results.append({'Metric':'ACC',split:accuracy})
    classificatio_results.append({'Metric':'Macro-F1',split:f1_macro})
    classificatio_results.append({'Metric':'Micro-F1',split:f1_micro})
    classificatio_results.append({'Metric':'Macro-P',split:precision_macro})
    classificatio_results.append({'Metric':'Micro-P',split:precision_micro})
    classificatio_results.append({'Metric':'Macro-R',split:recall_macro})
    classificatio_results.append({'Metric':'Micro-R',split:recall_macro})
    classificatio_results.append({'Metric':'MCC',split:mcc})
    results = pd.DataFrame(classificatio_results)

    results[split]=results[split].apply(lambda x: round(x*100, 2))
    return results

def evaluate_llm_binary_classification(prompt_id=Literal['prompt_1', 'prompt_2', 'prompt_3', 'prompt_4']):
    df = pd.read_excel(f"{path_to_results}/{prompt_id}/evaluation_results_all.xlsx")
    # ALL
    y_true = df.label.tolist()
    y_true = [1 if x=='has_criteria' else 0 for x in y_true]
    y_pred = df.label_predicted.tolist()
    y_pred = [1 if x=='has_criteria' else 0 for x in y_pred]
    
    results_all = get_result_table(y_true, y_pred, 'All')
    
    # Validation
    validation_contexts = dataset['validation'].filename.tolist()
    df_filtered = df[df.filename.isin(validation_contexts)]
    y_true = df_filtered.label.tolist()
    y_true = [1 if x=='has_criteria' else 0 for x in y_true]
    y_pred = df_filtered.label_predicted.tolist()
    y_pred = [1 if x=='has_criteria' else 0 for x in y_pred]
    results_validation = get_result_table(y_true, y_pred, 'Validation')
    del results_validation['Metric']
    
    # Test
    test_contexts = dataset['test'].filename.tolist()
    df_filtered = df[df.filename.isin(test_contexts)]
    y_true = df_filtered.label.tolist()
    y_true = [1 if x=='has_criteria' else 0 for x in y_true]
    y_pred = df_filtered.label_predicted.tolist()
    y_pred = [1 if x=='has_criteria' else 0 for x in y_pred]
    results_test = get_result_table(y_true, y_pred, 'Test')
    del results_test['Metric']
    
    results_combined = pd.concat([results_validation, results_test, results_all ],axis=1)
    results_combined = results_combined[['Metric', 'Validation', 'Test', 'All']]
    return results_combined
    

In [1]:
llm_binary_clf_results = evaluate_llm_binary_classification('prompt_1')
llm_binary_clf_results

NameError: name 'evaluate_llm_binary_classification' is not defined

In [50]:
print(llm_binary_clf_results.to_latex(index=False, float_format="{:.2f}".format))

\begin{tabular}{lrrr}
\toprule
Metric & Validation & Test & All \\
\midrule
ACC & 91.47 & 93.08 & 92.07 \\
Macro-F1 & 91.37 & 93.04 & 92.01 \\
Micro-F1 & 91.47 & 93.08 & 92.07 \\
Macro-P & 92.86 & 93.64 & 93.09 \\
Micro-P & 91.47 & 93.08 & 92.07 \\
Macro-R & 92.86 & 93.64 & 93.09 \\
Micro-R & 92.86 & 93.64 & 93.09 \\
MCC & 84.11 & 86.63 & 85.02 \\
\bottomrule
\end{tabular}



# Named Entity Recognition

In [51]:
def make_ner_overview(averages, scores):
    # averages = ['macro']
    # scores = ['f1']

    all_results = list()

    for prompt_id in prompt_id_list:
        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_all.xlsx")

        span_scores = df.span_scores.tolist()
        span_scores = [literal_eval(x) for x in span_scores]


        for average in averages:
            for m in scores:
                metrics =  dict()
                m = m+'_'+average
                key = mapping[m]
                metrics['Prompt']=mapping[prompt_id]
                metrics['Criteria Presence']='mixed'
                metrics['Metric']=key
                for label in ['kriterium', 'gewichtung', 'maxPunkte', 'zkNummer']:
                    hmean = mean([x[label][m] for x in span_scores])*100
                    hmean = round(hmean,2)
                    metrics[mapping[label]]= hmean
                    all_results.append(metrics)


        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_has_criteria.xlsx")

        span_scores = df.span_scores.tolist()
        span_scores = [literal_eval(x) for x in span_scores]


        for average in averages:
            for m in scores:
                metrics =  dict()
                m = m+'_'+average
                key = mapping[m]
                metrics['Prompt']=mapping[prompt_id]
                metrics['Criteria Presence']='with ACs only'
                metrics['Metric']=key
                for label in ['kriterium', 'gewichtung', 'maxPunkte', 'zkNummer']:
                    hmean = mean([x[label][m] for x in span_scores])*100
                    hmean = round(hmean,2)
                    metrics[mapping[label]]= hmean
                    all_results.append(metrics)
                    
        
        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_has_no_criteria.xlsx")

        span_scores = df.span_scores.tolist()
        span_scores = [literal_eval(x) for x in span_scores]


        for average in averages:
            for m in scores:
                metrics =  dict()
                m = m+'_'+average
                key = mapping[m]
                metrics['Prompt']=mapping[prompt_id]
                metrics['Criteria Presence']='without ACs only'
                metrics['Metric']=key
                for label in ['kriterium', 'gewichtung', 'maxPunkte', 'zkNummer']:
                    hmean = mean([x[label][m] for x in span_scores])*100
                    hmean = round(hmean,2)
                    metrics[mapping[label]]= hmean
                    all_results.append(metrics)


    all_results_df = pd.DataFrame(all_results).sort_values(['Criteria Presence', 'Prompt']).drop_duplicates()
    all_results_df = all_results_df[['Prompt', 'Criteria Presence', 'Metric', 'AC', 'ID', 'MNP', 'W', ]]
    all_results_ner = bold_max_values(all_results_df)
    return all_results_ner

all_results_ner_combined = []
for score in scores:
    for average in averages:
        res = make_ner_overview(averages=[average], scores=[score])
        all_results_ner_combined.append(res)


all_results_ner_combined = pd.concat(all_results_ner_combined, axis=0)

# del all_results_ner_combined['\\textbf{Metric}']

#all_results_ner_combined = all_results_ner_combined.sort_values(['\\textbf{Criteria Presence}', '\\textbf{Prompt}'])

print(all_results_ner_combined.to_latex(index=False, float_format="{:.2f}".format))
 

\begin{tabular}{lllllll}
\toprule
\textbf{Prompt} & \textbf{Criteria Presence} & \textbf{Metric} & \textbf{AC} & \textbf{ID} & \textbf{MNP} & \textbf{W} \\
\midrule
1 & mixed & Macro-P & 34.04 & 36.77 & 41.97 & 44.53 \\
2 & mixed & Macro-P & 37.35 & 38.23 & 44.86 & 48.20 \\
3 & mixed & Macro-P & 37.68 & 40.67 & 43.07 & 26.81 \\
4 & mixed & Macro-P & 43.89 & 47.82 & 52.93 & 55.11 \\
1 & with ACs only & Macro-P & 64.45 & 69.83 & 80.03 & 85.07 \\
2 & with ACs only & Macro-P & \textbf{64.98} & 66.71 & 79.73 & 86.29 \\
3 & with ACs only & Macro-P & 60.83 & 66.10 & 66.02 & 34.09 \\
4 & with ACs only & Macro-P & 64.64 & \textbf{72.36} & \textbf{82.39} & \textbf{86.69} \\
1 & without ACs only & Macro-P & 2.48 & 2.48 & 2.48 & 2.48 \\
2 & without ACs only & Macro-P & 8.70 & 8.70 & 8.70 & 8.70 \\
3 & without ACs only & Macro-P & 13.66 & 14.29 & 19.25 & 19.25 \\
4 & without ACs only & Macro-P & 22.36 & 22.36 & 22.36 & 22.36 \\
1 & mixed & Micro-P & 40.53 & 40.05 & 43.22 & 48.26 \\
2 & mixed & Micr

In [52]:
all_results_ner_combined

,\textbf{Prompt},\textbf{Criteria Presence},\textbf{Metric},\textbf{AC},\textbf{ID},\textbf{MNP},\textbf{W}
0,1,mixed,Macro-P,34.04,36.77,41.97,44.53
12,2,mixed,Macro-P,37.35,38.23,44.86,48.2
24,3,mixed,Macro-P,37.68,40.67,43.07,26.81
36,4,mixed,Macro-P,43.89,47.82,52.93,55.11
4,1,with ACs only,Macro-P,64.45,69.83,80.03,85.07
...,...,...,...,...,...,...,...
40,4,with ACs only,Micro-F1,72.62,\textbf{75.4},\textbf{85.31},\textbf{90.78}
8,1,without ACs only,Micro-F1,2.48,2.48,2.48,2.48
20,2,without ACs only,Micro-F1,8.7,8.7,8.7,8.7
32,3,without ACs only,Micro-F1,13.66,14.29,19.25,19.25


# Relation Extraction

In [53]:
def make_re_overview(averages, scores):
    all_results = list()

    for prompt_id in prompt_id_list:
        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_all.xlsx")
        column_selected = df.key_value_eval.tolist()
        key_value_eval = list()

        for entry in column_selected:
            entry = literal_eval(entry)
            key_value_eval.append(entry)

        with_criteria='mixed'
        for mean_method in ['mean']:
            metrics =  dict()
            metrics['Prompt']=mapping[prompt_id]
            metrics['Criteria Presence']=with_criteria
            for average in averages:
                for m in scores:
                    m = m+'_'+average
                    key = mapping[m]
                    if mean_method == 'mean':
                        hmean = mean([x[m] for x in key_value_eval])*100
                        hmean = round(hmean,2)
                        metrics[key]= hmean
            all_results.append(metrics)

        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_has_criteria.xlsx")
        column_selected = df.key_value_eval.tolist()
        key_value_eval = list()

        for entry in column_selected:
            entry = literal_eval(entry)
            key_value_eval.append(entry)

        with_criteria='with criteria only'
        for mean_method in ['mean']:
            metrics =  dict()
            metrics['Prompt']=mapping[prompt_id]
            metrics['Criteria Presence']=with_criteria
            for average in averages:
                for m in scores:
                    m = m+'_'+average
                    key = mapping[m]
                    if mean_method == 'mean':
                        hmean = mean([x[m] for x in key_value_eval])*100
                        hmean = round(hmean,2)
                        metrics[key]= hmean
            all_results.append(metrics)

        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_has_no_criteria.xlsx")
        column_selected = df.key_value_eval.tolist()
        key_value_eval = list()

        for entry in column_selected:
            entry = literal_eval(entry)
            key_value_eval.append(entry)

        with_criteria='without criteria only'
        for mean_method in ['mean']:
            metrics =  dict()
            metrics['Prompt']=mapping[prompt_id]
            metrics['Criteria Presence']=with_criteria
            for average in averages:
                for m in scores:
                    m = m+'_'+average
                    key = mapping[m]
                    if mean_method == 'mean':
                        hmean = mean([x[m] for x in key_value_eval])*100
                        hmean = round(hmean,2)
                        metrics[key]= hmean
            all_results.append(metrics)

    all_results_df = pd.DataFrame(all_results).sort_values(['Criteria Presence', 'Prompt'])
    all_results_df = bold_max_values(all_results_df)
    return all_results_df

all_results_re_macro = make_re_overview(averages=['macro'], scores=['f1'])
all_results_re_micro = make_re_overview(averages=['micro'], scores=['f1'])


all_results_re_combined = pd.concat([all_results_re_macro, all_results_re_micro], axis=1)
all_results_re_combined = all_results_re_macro.merge(all_results_re_micro, on=['\\textbf{Prompt}', '\\textbf{Criteria Presence}'])

# del all_results_re_macro['\\textbf{Metric}']

#all_results_ner_combined = all_results_ner_combined.sort_values(['\\textbf{Criteria Presence}', '\\textbf{Prompt}'])

print(all_results_re_combined.to_latex(index=False, float_format="{:.2f}".format))
 


\begin{tabular}{llll}
\toprule
\textbf{Prompt} & \textbf{Criteria Presence} & \textbf{Macro-F1} & \textbf{Micro-F1} \\
\midrule
1 & mixed & 21.00 & 24.19 \\
2 & mixed & 23.76 & 26.74 \\
3 & mixed & 13.51 & 14.79 \\
4 & mixed & 33.15 & 36.35 \\
1 & with criteria only & 39.46 & 45.71 \\
2 & with criteria only & 38.28 & 44.14 \\
3 & with criteria only & 13.36 & 15.87 \\
4 & with criteria only & \textbf{43.56} & \textbf{49.84} \\
1 & without criteria only & 1.86 & 1.86 \\
2 & without criteria only & 8.70 & 8.70 \\
3 & without criteria only & 13.66 & 13.66 \\
4 & without criteria only & 22.36 & 22.36 \\
\bottomrule
\end{tabular}



In [54]:
all_results_re_combined

,\textbf{Prompt},\textbf{Criteria Presence},\textbf{Macro-F1},\textbf{Micro-F1}
0,1,mixed,21.0,24.19
1,2,mixed,23.76,26.74
2,3,mixed,13.51,14.79
3,4,mixed,33.15,36.35
4,1,with criteria only,39.46,45.71
5,2,with criteria only,38.28,44.14
6,3,with criteria only,13.36,15.87
7,4,with criteria only,\textbf{43.56},\textbf{49.84}
8,1,without criteria only,1.86,1.86
9,2,without criteria only,8.7,8.7


# Formatting

In [55]:
def make_formatting_overview():
    all_results = list()

    for prompt_id in prompt_id_list:

        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_all.xlsx")
        entry = {
                'Prompt':mapping[prompt_id],
                'Criteria Presence': 'mixed',
                'Successful':0,
                'Failed':0
                }
        for row in df.to_dict(orient='records'):
            if row['conversion_status']=='ok':
                entry['Successful']+=1
            if row['conversion_status']=='failed':
                entry['Failed']+=1
        all_results.append(entry)
        
        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_has_criteria.xlsx")
        entry = {
                'Prompt':mapping[prompt_id],
                'Criteria Presence': 'with criteria only',
                'Successful':0,
                'Failed':0
                }
        for row in df.to_dict(orient='records'):
            if row['conversion_status']=='ok':
                entry['Successful']+=1
            if row['conversion_status']=='failed':
                entry['Failed']+=1
        all_results.append(entry)
        
        
        df = pd.read_excel(path_to_results+f"{prompt_id}/evaluation_results_has_no_criteria.xlsx")
        entry = {
                'Prompt':mapping[prompt_id],
                'Criteria Presence': 'without criteria only',
                'Successful':0,
                'Failed':0
                }
        for row in df.to_dict(orient='records'):
            if row['conversion_status']=='ok':
                entry['Successful']+=1
            if row['conversion_status']=='failed':
                entry['Failed']+=1
        all_results.append(entry)
    

    
    all_results_df = pd.DataFrame(all_results).sort_values(['Criteria Presence', 'Prompt'])
    all_results_df = make_columns_bold(all_results_df)
    return all_results_df
    
    
    
            
print(make_formatting_overview().to_latex(index=False, float_format="{:.2f}".format))   

\begin{tabular}{llrr}
\toprule
\textbf{Prompt} & \textbf{Criteria Presence} & \textbf{Successful} & \textbf{Failed} \\
\midrule
1 & mixed & 327 & 1 \\
2 & mixed & 327 & 1 \\
3 & mixed & 328 & 0 \\
4 & mixed & 328 & 0 \\
1 & with criteria only & 167 & 0 \\
2 & with criteria only & 167 & 0 \\
3 & with criteria only & 167 & 0 \\
4 & with criteria only & 167 & 0 \\
1 & without criteria only & 160 & 1 \\
2 & without criteria only & 160 & 1 \\
3 & without criteria only & 161 & 0 \\
4 & without criteria only & 161 & 0 \\
\bottomrule
\end{tabular}



In [56]:
make_formatting_overview()

,\textbf{Prompt},\textbf{Criteria Presence},\textbf{Successful},\textbf{Failed}
0,1,mixed,327,1
3,2,mixed,327,1
6,3,mixed,328,0
9,4,mixed,328,0
1,1,with criteria only,167,0
4,2,with criteria only,167,0
7,3,with criteria only,167,0
10,4,with criteria only,167,0
2,1,without criteria only,160,1
5,2,without criteria only,160,1
